<a href="https://colab.research.google.com/github/AAKAAASSHHH24/MLOPS-_WANDB/blob/main/split_the_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preparation

In this notebook we will prepare the data to later train our deep learning model. To do so,

we will start a new W&B run and use our raw data artifact

split the data and save the splits into a new W&B Artifact

join information about the split with our EDA Table

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install Weights & Biases logging library
!pip install wandb -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 KB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 21.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.9 MB/s eta 0:00:00


In [5]:
import os
ROOT = '/content/drive/MyDrive/WnB'
os.chdir(ROOT)

In [6]:

import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [7]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Let's use artifact we previously saved to W&B (we're storing artifact names and other global parameters in params).

In [8]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())
     

wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:1:18.9


In [9]:
path.ls()

(#5) [Path('artifacts/bdd_simple_1k:v0/images'),Path('artifacts/bdd_simple_1k:v0/LICENSE.txt'),Path('artifacts/bdd_simple_1k:v0/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v0/labels'),Path('artifacts/bdd_simple_1k:v0/media')]

To split data between training, testing and validation, we need file names, groups (derived from the file name) and target (here we use our rare class bicycle for stratification). We previously saved these columns to EDA table, so let's retrieve it from the table now.

In [10]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [11]:
orig_eda_table = raw_data_at.get("eda_table")


wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:45.4


In [12]:
y = orig_eda_table.get_column('bicycle')

Now we will split the data into train (80%), validation (10%) and test (10%) sets. As we do that, we need to be careful to:

avoid leakage: for that reason we are grouping data according to video identifier (we want to make sure our model can generalize to new cars or video frames)

handle the label imbalance: for that reason we stratify data with our target column

We will use sklearn's StratifiedGroupKFold to split the data into 10 folds and assign 1 fold for test, 1 for validation and the rest for training.

In [13]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [14]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i
     

In [15]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()


train    800
valid    100
test     100
Name: Stage, dtype: int64

In [16]:
df.to_csv('data_split.csv', index=False)

We will now create a new artifact and add our data there.

In [17]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [18]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v0)... Done. 7.9s


Finally, the split information may be relevant for our analyses - rather than uploading images again, we will save the split information to a new table and join it with EDA table we created previously.

In [19]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']]) # using a dataframe to create a wandb table

In [20]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

Let's add it to our artifact, log it and finish our run.

In [21]:
processed_data_at.add(join_table, "eda_table_data_split")

<ManifestEntry digest: 6DG0auTjOyPUIFOMGa2lHA==>

In [22]:
run.log_artifact(processed_data_at)
run.finish()